In [52]:
try:
    import modin.pandas as pd # multiprocessing pandas
except ImportError:
    import pandas as pd

IMPORT DATA

In [53]:
db = read_csv("../data/collegeinsight_data_nolabel_ICs_by_year.csv", encoding= 'unicode_escape')
db

CLEAN DATA

In [ ]:
#db[['data_yr_string', 'name']]

pred_data = ['coa_tuit_fees_d', 'coa_books_supp_d', 'coa_on_room_board_d']

db['name'] = db['name'].str.upper() # make all names uppercase

db = db[['name', 'data_yr_string'] + pred_data] # select only the columns we need

for dat in pred_data:
    db = db[db[dat].notnull()] # remove rows with null values


yrs = sorted(list(set(db['data_yr_string'])))
print(sorted(yrs))
colleges = list(db[db['data_yr_string'] == yrs[0]]['name'])

#print(colleges)
print([("YALE" in x) for x in colleges if "YALE" in x])

for yr in yrs[1:]:
    col = list(db[db['data_yr_string'] == yr]['name'])

    colleges = [x for x in colleges if x in col]


print(len(colleges))

db = db[[x in colleges for x in db['name']]]


#db[db['data_yr_string'] == yrs[0] and db['name'] == "YALE UNIVERSITY"]



['2000-01', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19']
[]
1453


Check entries

In [ ]:
db[db['name'] == "UNIVERSITY OF CONNECTICUT"]

,name,data_yr_string,coa_tuit_fees_d,coa_books_supp_d,coa_on_room_board_d
1946,UNIVERSITY OF CONNECTICUT,2000-01,5596.0,725.0,6062.0
11462,UNIVERSITY OF CONNECTICUT,2003-04,6800.0,725.0,6888.0
19207,UNIVERSITY OF CONNECTICUT,2004-05,7490.0,725.0,7404.0
26929,UNIVERSITY OF CONNECTICUT,2005-06,7912.0,725.0,7848.0
34682,UNIVERSITY OF CONNECTICUT,2006-07,8362.0,726.0,8266.0
42440,UNIVERSITY OF CONNECTICUT,2007-08,8852.0,726.0,8850.0
50336,UNIVERSITY OF CONNECTICUT,2008-09,9338.0,800.0,9504.0
58324,UNIVERSITY OF CONNECTICUT,2009-10,9886.0,800.0,10120.0
66593,UNIVERSITY OF CONNECTICUT,2010-11,10416.0,800.0,10782.0
74990,UNIVERSITY OF CONNECTICUT,2011-12,10670.0,850.0,11050.0


ML  MODELING

In [ ]:
from keras import layers, Sequential, models

In [ ]:
import warnings
warnings.simplefilter(action='ignore')

years = [int(x.split("-")[0]) for x in yrs]

columns = ["name"] + years
print(columns)

tuition_db = pd.DataFrame(columns = columns)
tuition_db['name'] = colleges

supplies_db = tuition_db.copy()

room_db = tuition_db.copy()


for i in range(len(colleges)):
    tuition = [db[db['name'] == colleges[i]][db['data_yr_string'] == x]['coa_tuit_fees_d'].values[0] for x in yrs]
    supplies = [db[db['name'] == colleges[i]][db['data_yr_string'] == x]['coa_books_supp_d'].values[0] for x in yrs]
    room = [db[db['name'] == colleges[i]][db['data_yr_string'] == x]['coa_on_room_board_d'].values[0] for x in yrs]
    #print([college] + tuition)
    #row = pd.DataFrame([colleges[i]] + tuition, columns = columns)
    #print(row)
    
    for yr in years:
        tuition_db.loc[i, yr] = tuition[years.index(yr)]
        supplies_db.loc[i, yr] = supplies[years.index(yr)]
        room_db.loc[i, yr] = room[years.index(yr)]

    print(f"{i+1} / {len(colleges)}", end = "\r")

    #print(tuition)
    #print("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAa")
    #tuition_db[tuition_db['name'] == colleges[i]] = tuition[i]
    #tuition_db[i] = tuition

    
#print(tuition_db)


['name', 2000, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]


KeyboardInterrupt: 

Save the dataframes to csv files

In [ ]:
tuition_db.to_csv("../data/tuition_db.csv")
supplies_db.to_csv("../data/supplies_db.csv")
room_db.to_csv("../data/room_db.csv")

Load the dataframes from csv files

In [ ]:
tuition_db = read_csv("../data/tuition_db.csv")
supplies_db = read_csv("../data/supplies_db.csv")
room_db = read_csv("../data/room_db.csv")